In [1]:
import numpy as np
import pandas as pd
import itertools
import time
from random import random
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import (MultiprocessingEvaluator, 
                           Scenario, Constraint, Policy,
                           ScalarOutcome)
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.em_framework.evaluators import (perform_experiments, optimize,
                                                   BaseEvaluator)
from ema_workbench.util import ema_logging, save_results, load_results
from ema_workbench.analysis import (pairs_plotting, prim, 
                                    feature_scoring, parcoords,
                                    dimensional_stacking)

from problem_formulation import get_model_for_problem_formulation
%matplotlib inline

ema_logging.log_to_stderr(ema_logging.INFO)

BaseEvaluator.reporting_frequency = 0.01

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
dike_model,planning_steps = get_model_for_problem_formulation(5)

In [3]:
MORDM_levers = load_results('scenario1 MORDMruns_worst case scenario.tar.gz')
MORDM_pol,outcomes = MORDM_levers
MORDM_pol = MORDM_pol.sample(n=3)
MORDM_pol = MORDM_pol.iloc[:,0:21]
#results_df = pd.DataFrame.from_dict(_) 

[MainProcess/INFO] results loaded succesfully from C:\Users\user\EPA1361\fin_ass\Runs on sollution\scenario1 MORDMruns_worst case scenario.tar.gz


In [4]:
MORO_res= pd.read_csv('moro_results.xls')
MORO_res = MORO_res.sample(n=3)
MORO_pol = MORO_res.iloc[:,1:22]

In [5]:
fin_levers = pd.concat([MORDM_pol,MORO_pol])
fin_levers = fin_levers.reset_index()
fin_levers = fin_levers.rename(index={0:'MORDM1',1:'MORDM2',2:'MORDM3',3:'MORO1',
                                     4:'MORO2',5:'MORO3'})
fin_levers = fin_levers.drop(['index'],axis=1)

In [6]:
policies = []
pol0 = {}
for key in dike_model.levers:
    pol0.update({key.name:0})                
policy0 = Policy('Policy 0', **pol0)
policies.append(policy0)
for i,lever in enumerate(fin_levers.index):
    policy = fin_levers.loc[lever].to_dict()
    policy = Policy(fin_levers.index[i], **policy)
    policies.append(policy)

In [8]:
n_scenarios = 2500

#start = time.time()
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies = policies)
    end = time.time()
#print(round((end - start)/3600, 2), "Hours")
save_results(results, "random7pol.tar.gz")

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2500 scenarios * 7 policies * 1 model(s) = 17500 experiments
[MainProcess/INFO] 1750 cases completed
[MainProcess/INFO] 3500 cases completed
[MainProcess/INFO] 5250 cases completed
[MainProcess/INFO] 7000 cases completed
[MainProcess/INFO] 8750 cases completed
[MainProcess/INFO] 10500 cases completed
[MainProcess/INFO] 12250 cases completed
[MainProcess/INFO] 14000 cases completed
[MainProcess/INFO] 15750 cases completed
[MainProcess/INFO] 17500 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] results saved successfully to C:\Users\user\EPA1361\fin_ass\Runs on sollution\random7pol.tar.gz
